In [2]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor

torch.manual_seed(1)
class PosDatasetReader(DatasetReader):
    """
    DatasetReader for PoS tagging data, one sentence per line, like

        The###DET dog###NN ate###V the###DET apple###NN
    """
    def __init__(self, token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
    def text_to_instance(self, tokens: List[Token], tags: List[str] = None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"sentence": sentence_field}

        if tags:
            label_field = SequenceLabelField(labels=tags, sequence_field=sentence_field)
            fields["labels"] = label_field

        return Instance(fields)
    def _read(self, file_path: str) -> Iterator[Instance]:
        with open(file_path) as f:
            for line in f:
                pairs = line.strip().split()
                sentence, tags = zip(*(pair.split("###") for pair in pairs))
                yield self.text_to_instance([Token(word) for word in sentence], tags)
class LstmTagger(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()
    def forward(self,
                sentence: Dict[str, torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str, torch.Tensor]:
        mask = get_text_field_mask(sentence)
        embeddings = self.word_embeddings(sentence)
        encoder_out = self.encoder(embeddings, mask)
        tag_logits = self.hidden2tag(encoder_out)
        output = {"tag_logits": tag_logits}
        if labels is not None:
            self.accuracy(tag_logits, labels, mask)
            output["loss"] = sequence_cross_entropy_with_logits(tag_logits, labels, mask)

        return output
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}
reader = PosDatasetReader()
train_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/training.txt'))
validation_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/validation.txt'))
vocab = Vocabulary.from_instances(train_dataset + validation_dataset)
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("sentence", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=1000,
                  cuda_device=cuda_device)

2it [00:00, 2006.84it/s]
2it [00:00, 2005.88it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 4163.08it/s]


In [3]:
trainer.train()

accuracy: 0.3333, loss: 1.1685 ||: 100%|█████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.19it/s]
accuracy: 0.3333, loss: 1.1592 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.65it/s]
accuracy: 0.3333, loss: 1.1604 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.69it/s]
accuracy: 0.3333, loss: 1.1516 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.84it/s]
accuracy: 0.3333, loss: 1.1529 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 143.25it/s]
accuracy: 0.3333, loss: 1.1445 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.54it/s]
accuracy: 0.3333, loss: 1.1458 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.57it/s]
accuracy: 0.3333, loss: 1.1379 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.69it/s]
accuracy: 0.3333, loss: 1.1391 ||: 100%|

accuracy: 0.4444, loss: 1.0479 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.68it/s]
accuracy: 0.4444, loss: 1.0463 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.21it/s]
accuracy: 0.4444, loss: 1.0477 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.55it/s]
accuracy: 0.4444, loss: 1.0460 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.10it/s]
accuracy: 0.4444, loss: 1.0474 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.69it/s]
accuracy: 0.4444, loss: 1.0457 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.99it/s]
accuracy: 0.4444, loss: 1.0471 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.71it/s]
accuracy: 0.4444, loss: 1.0454 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.05it/s]
accuracy: 0.4444, loss: 1.0468 ||: 100%|

accuracy: 0.4444, loss: 1.0173 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.14it/s]
accuracy: 0.4444, loss: 1.0156 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.31it/s]
accuracy: 0.4444, loss: 1.0166 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.50it/s]
accuracy: 0.4444, loss: 1.0149 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.62it/s]
accuracy: 0.4444, loss: 1.0159 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.06it/s]
accuracy: 0.4444, loss: 1.0141 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.75it/s]
accuracy: 0.4444, loss: 1.0151 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 143.28it/s]
accuracy: 0.4444, loss: 1.0134 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.21it/s]
accuracy: 0.4444, loss: 1.0144 ||: 100%|

accuracy: 0.4444, loss: 0.9270 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.15it/s]
accuracy: 0.4444, loss: 0.9242 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.69it/s]
accuracy: 0.5556, loss: 0.9248 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.56it/s]
accuracy: 0.4444, loss: 0.9220 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.66it/s]
accuracy: 0.5556, loss: 0.9225 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.52it/s]
accuracy: 0.4444, loss: 0.9197 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.07it/s]
accuracy: 0.5556, loss: 0.9203 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.12it/s]
accuracy: 0.4444, loss: 0.9175 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.29it/s]
accuracy: 0.5556, loss: 0.9180 ||: 100%|

accuracy: 0.6667, loss: 0.7148 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.10it/s]
accuracy: 0.6667, loss: 0.7119 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.66it/s]
accuracy: 0.6667, loss: 0.7112 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.54it/s]
accuracy: 0.6667, loss: 0.7082 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.15it/s]
accuracy: 0.6667, loss: 0.7075 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.10it/s]
accuracy: 0.6667, loss: 0.7046 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.31it/s]
accuracy: 0.6667, loss: 0.7039 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.08it/s]
accuracy: 0.6667, loss: 0.7010 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.62it/s]
accuracy: 0.6667, loss: 0.7002 ||: 100%|

...

accuracy: 1.0000, loss: 0.0352 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.44it/s]
accuracy: 1.0000, loss: 0.0350 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.29it/s]
accuracy: 1.0000, loss: 0.0351 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.11it/s]
accuracy: 1.0000, loss: 0.0349 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.35it/s]
accuracy: 1.0000, loss: 0.0349 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.64it/s]
accuracy: 1.0000, loss: 0.0348 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.29it/s]
accuracy: 1.0000, loss: 0.0348 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.14it/s]
accuracy: 1.0000, loss: 0.0347 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.05it/s]
accuracy: 1.0000, loss: 0.0347 ||: 100%|

accuracy: 1.0000, loss: 0.0284 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 166.73it/s]
accuracy: 1.0000, loss: 0.0282 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.10it/s]
accuracy: 1.0000, loss: 0.0283 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.77it/s]
accuracy: 1.0000, loss: 0.0282 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.18it/s]
accuracy: 1.0000, loss: 0.0282 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.56it/s]
accuracy: 1.0000, loss: 0.0281 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.18it/s]
accuracy: 1.0000, loss: 0.0281 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.51it/s]
accuracy: 1.0000, loss: 0.0280 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.66it/s]
accuracy: 1.0000, loss: 0.0280 ||: 100%|

accuracy: 1.0000, loss: 0.0236 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.09it/s]
accuracy: 1.0000, loss: 0.0235 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.31it/s]
accuracy: 1.0000, loss: 0.0235 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.55it/s]
accuracy: 1.0000, loss: 0.0234 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.15it/s]
accuracy: 1.0000, loss: 0.0234 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.44it/s]
accuracy: 1.0000, loss: 0.0234 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.13it/s]
accuracy: 1.0000, loss: 0.0234 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.63it/s]
accuracy: 1.0000, loss: 0.0233 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.05it/s]
accuracy: 1.0000, loss: 0.0233 ||: 100%|

accuracy: 1.0000, loss: 0.0200 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.52it/s]
accuracy: 1.0000, loss: 0.0200 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.23it/s]
accuracy: 1.0000, loss: 0.0200 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 167.13it/s]
accuracy: 1.0000, loss: 0.0199 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.10it/s]
accuracy: 1.0000, loss: 0.0199 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 187.60it/s]
accuracy: 1.0000, loss: 0.0199 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.15it/s]
accuracy: 1.0000, loss: 0.0199 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.52it/s]
accuracy: 1.0000, loss: 0.0198 ||: 100%|████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.91it/s]
accuracy: 1.0000, loss: 0.0198 ||: 100%|

{'best_epoch': 999,
 'peak_cpu_memory_MB': 0,
 'training_duration': '0:00:21.048774',
 'training_start_epoch': 0,
 'training_epochs': 999,
 'epoch': 999,
 'training_accuracy': 1.0,
 'training_loss': 0.01809464395046234,
 'training_cpu_memory_MB': 0.0,
 'validation_accuracy': 1.0,
 'validation_loss': 0.018039997667074203,
 'best_validation_accuracy': 1.0,
 'best_validation_loss': 0.018039997667074203}

In [10]:
import os

In [11]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
tag_logits = predictor.predict("The dog ate the apple")['tag_logits']
tag_ids = np.argmax(tag_logits, axis=-1)
print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
# Here's how to save the model.
with open(os.path.join("tmp", "model.th"), 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files(os.path.join("tmp", "vocabulary"))
# And here's how to reload the model.
vocab2 = Vocabulary.from_files(os.path.join("tmp", "vocabulary"))
model2 = LstmTagger(word_embeddings, lstm, vocab2)
with open(os.path.join("tmp", "model.th"), 'rb') as f:
    model2.load_state_dict(torch.load(f))
if cuda_device > -1:
    model2.cuda(cuda_device)

['DET', 'NN', 'V', 'DET', 'NN']


In [21]:
predictor2 = SentenceTaggerPredictor(model2, dataset_reader=reader)
tag_logits2 = predictor2.predict("The dog ate the apple and ate the cat")['tag_logits']
# np.testing.assert_array_almost_equal(tag_logits2, tag_logits)
tag_ids2 = np.argmax(tag_logits2, axis=-1)
print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids2])

['DET', 'NN', 'V', 'DET', 'NN', 'NN', 'V', 'DET', 'NN']
